<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/MNIST_noHiddden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.ops.gen_math_ops import Cross
from tensorflow.python.eager.backprop import GradientTape
import tensorflow as tf
from tqdm import tqdm

class LinearRegression(tf.keras.Model):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.one_layer = tf.keras.layers.Dense(10,
                                           activation=None,
                                           kernel_initializer='zeros', # weight 값
                                           bias_initializer= 'zeros')
    def call(self, x): # Forward
        y1 = self.one_layer(x)
        return tf.nn.softmax(y1)

@tf.function
def cross_entropy(y, d):
    return tf.reduce_mean(-tf.reduce_sum(d*tf.math.log(y), axis=[1]))


@tf.function
def backward(model, optimizers, x, y):
    with tf.GradientTape() as gt:
        y_pred = model(x)
        loss = cross_entropy(y_pred, y)

    grad = gt.gradient(loss, model.trainable_variables)
    optimizers.apply_gradients(zip(grad, model.trainable_variables))

@tf.function
def pred_accuracy(y_pred, y):
    correction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correction, tf.float32))
    return accuracy

@tf.function
def train_learning(eps, tdi, mod, opt, xtest, ytest):
    for epoch in tqdm(range(eps)):
        bx, by = next(tdi)
        backward(mod, opt, bx, by)
    accuracy = pred_accuracy(model(xtest), ytest)
    return accuracy

In [ ]:
if __name__ == '__main__':
    (X_train,  y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    print(X_train.shape)
    X_train, X_test = X_train.astype('float32'), X_test.astype('float32')
    X_train, X_test = X_train.reshape([-1, 784]), X_test.reshape([-1, 784])
    X_train, X_test = X_train/255.0, X_test/255. # 정규화
    y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)) #데이터묶음
    train_data = train_data.repeat().shuffle(60000).batch(100)
    train_data_iter = iter(train_data)
    optimizers = tf.optimizers.SGD(0.5)

    model = LinearRegression()
    epochs = 1000
    accuracy = train_learning(epochs, train_data_iter, model, optimizers, X_test, y_test)
    print(f"Accuracy: {accuracy}")